# Import CSV file containing county level census data from 2010

In [74]:
import pandas as pd 
import numpy as np
!conda install -c anaconda xlrd --yes


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [85]:
#data downloaded from https://factfinder.census.gov/ 
county_2010 = pd.read_excel("DEC_10_SF1_GCTPH1.US05PR.xlsx")
county_2010.head()


,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,0100000US,NaN,United States,0100000US,NaN,United States,United States,308745538,131704730,3796742.23,264836.79,3531905.43,87.4,37.3
1,0100000US,NaN,United States,0400000US01,1.0,United States - Alabama,Alabama,4779736,2171853,52420.07,1774.74,50645.33,94.4,42.9
2,0100000US,NaN,United States,0500000US01001,1001.0,United States - Alabama - Autauga County,Autauga County,54571,22135,604.39,9.95,594.44,91.8,37.2
3,0100000US,NaN,United States,0500000US01003,1003.0,United States - Alabama - Baldwin County,Baldwin County,182265,104061,2027.31,437.53,1589.78,114.6,65.5
4,0100000US,NaN,United States,0500000US01005,1005.0,United States - Alabama - Barbour County,Barbour County,27457,11829,904.52,19.64,884.88,31.0,13.4


## Limit Dataframe to Columns and Rows of Interest

In [86]:
#rename columns and keep only columns of interest
county_2010.rename(columns={"GCT_STUB.display-label": "state_county",
                            "GCT_STUB.target-geo-id": "county_geo_id",
                            "GCT_STUB.display-label.1": "county", 
                            "HD01": "population", 
                           "SUBHD0303": "land_sq_mile",
                           "SUBHD0401": "pop_density_sq_mile"}, inplace='True')
county2_2010 = county_2010[['state_county','county_geo_id','county','population','land_sq_mile','pop_density_sq_mile']].set_index('state_county')
county2_2010.head()

,county_geo_id,county,population,land_sq_mile,pop_density_sq_mile
state_county,,,,,
United States,0100000US,United States,308745538,3531905.43,87.4
United States - Alabama,0400000US01,Alabama,4779736,50645.33,94.4
United States - Alabama - Autauga County,0500000US01001,Autauga County,54571,594.44,91.8
United States - Alabama - Baldwin County,0500000US01003,Baldwin County,182265,1589.78,114.6
United States - Alabama - Barbour County,0500000US01005,Barbour County,27457,884.88,31.0


In [87]:
#limit to Polk and Dallas County in Iowa
iowa_county_2010 = county2_2010.filter(like='Iowa',axis=0)
pd_county_2010 = iowa_county_2010.set_index('county').loc[['Polk County','Dallas County']]
pd_county_2010

,county_geo_id,population,land_sq_mile,pop_density_sq_mile
county,,,,
Polk County,0500000US19153,430640,573.79,750.5
Dallas County,0500000US19049,66135,588.45,112.4


In [82]:
# since the population estimate is from 2010, get newer population estimates
county_2018 = pd.read_excel("polk_dallas_2018_pop.xlsx")
county_2018.rename(columns={"GEO_ID": 'county_geo_id', "B01003_001E": 'population_2018'}, inplace='True')
county2_2018 = county_2018[['county_geo_id','population_2018']].loc[[1,2]]
county2_2018


,county_geo_id,population_2018
1,0500000US19153,487204
2,0500000US19049,90180


In [96]:
#join 2 dataframes and create new population density column for 2018
pd_county_2018 = pd_county_2010.set_index('county_geo_id',append=True).join(county2_2018.set_index('county_geo_id'))
pd_county_2018['pop_density_sq_mile_2018'] = pd_county_2018['population_2018'] / pd_county_2018['land_sq_mile']
pd_county_2018

,,population,land_sq_mile,pop_density_sq_mile,population_2018,pop_density_sq_mile_2018
county,county_geo_id,,,,,
Polk County,0500000US19153,430640,573.79,750.5,487204,849.098
Dallas County,0500000US19049,66135,588.45,112.4,90180,153.25
